In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import math
from torch.utils.data import Dataset
import pickle
import datetime
import pandas as pd

In [ ]:
if(torch.cuda.is_available()):
    dev = torch.device("cuda")
else:
    dev = torch.device("cpu")
print(dev)

cuda
cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv
training_data = []
with open('/content/drive/MyDrive/clean_data_paris_marseille.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        row[0] = float(row[0])
        if row[0]>90:
            row[0] = row[0]-360
        training_data.append([float(item) for item in row])

with open("/content/drive/MyDrive/objet_wind_data_2020.pickle", "rb") as f:
    data = pickle.load(f)
    wind_data = data['data']


In [ ]:
angles = np.arctan2(wind_data[:,:,:,:,1], wind_data[:,:,:,:,0])
angles = np.round(((angles*360/(2*np.pi))/45))+4
directions = [np.where(angles == i , 1, 0) for i in range(1, 9)]
for i in range(8):
    directions[i] = np.sum(directions[i], axis = 1)
directions = np.array(directions)
direction_data= np.where(directions > 0 , 1, 0)

In [ ]:
data_std = 1

def grid_time(t):
    t = t
    t = t/3600
    t = t + 1 + 4
    t = t - t%6
    t = t - 1
    return (t+1)/6

def grid_lat(l):
    l = l*2
    l = round(l)
    l = l+2
    l = l - l%5
    l = l/2
    l = l +90
    l = l*2

    return (l/5)%73

def grid_long(l):
    l = l*2
    l = round(l)
    l = l+2
    l = l - l%5

    return (l/5)%144
def int_array(arr):
    return [int(i) for i in arr]


def grid_array(arr):
    return int_array([grid_long(arr[0]),grid_lat(arr[1]),grid_time(arr[2]),int(arr[3])])

def get_winds_at_t(data,time,dtime):
    tensor = torch.tensor([data[time:time+dtime,:,:,:]], dtype=torch.float32).permute(1,0,5,2,3,4)
    return tensor


def input_label(training_data,dtime,i, goal_coord,data_winds, data_dirs):
    array = grid_array(training_data[i])
    time = array[2]
    long = array[0]
    lat = array[1]
    alt = array[3]
    dirs = data_dirs[:,time:time+dtime,:,:]
    dirs = np.transpose(dirs,(1,0,2,3))
    pos = np.zeros((dtime,1,dirs.shape[2],dirs.shape[3]))
    pos[0,0,int(grid_long(goal_coord[0])),int(grid_lat(goal_coord[1]))] = -data_std*1.3
    pos[0,0,long,lat] = data_std*1.3
    input = np.concatenate((dirs,pos),axis=1)
    action = np.arctan2(data_winds[time,alt,long,lat,1],data_winds[time,alt,long,lat,0])
    action = int(np.round(((action+np.pi)*360/(2*np.pi))/45)%8)

    return (input,action)


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, entries, dtime, goal_coord, wind_data, direction_data, transform=None, target_transform=None):
        self.entries = entries
        self.transform = transform
        self.target_transform = target_transform
        self.dtime = dtime
        self.goal_coord = goal_coord
        self.wind_data = wind_data
        self.direction_data = direction_data


    def __len__(self):
        return len(self.entries)

    def __getitem__(self, idx):
        inp,lab = input_label(self.entries,self.dtime,idx, self.goal_coord,self.wind_data, self.direction_data)
        inp = np.pad(inp,((0,0),(0,0),(2,2),(0,0)),'wrap')
        return inp, lab

In [ ]:
coords_destination = [5.3698, 43.2965]
trainset = CustomDataset(training_data,1,coords_destination,wind_data,direction_data)
batch_size = 32
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)


In [ ]:
a,b = next(iter(trainloader))
a = a.float().to(dev)
print(a.shape)


torch.Size([32, 1, 9, 148, 73])


In [ ]:
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dropout):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, stride, padding)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

        self.bn1.weight.data.fill_(1)
        self.bn1.bias.data.zero_()
        self.bn2.weight.data.fill_(1)
        self.bn2.bias.data.zero_()

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity
        out = self.relu(out)
        out = self.dropout(out)
        return out

class ProjectedResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dropout):
        super(ProjectedResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride, 1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, 2, padding)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        self.projector = nn.AvgPool2d(kernel_size, stride = 2 , padding = padding)
        self.Conv1x1 = nn.Conv2d(in_channels, out_channels, 1, 1, 0)
        self.bn1.weight.data.fill_(1)
        self.bn1.bias.data.zero_()
        self.bn2.weight.data.fill_(1)
        self.bn2.bias.data.zero_()

    def forward(self, x):
        identity = x

        identity = self.projector(x)
        identity =  self.Conv1x1(identity)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x += identity
        x = self.relu(x)
        x = self.dropout(x)
        return x

In [ ]:
class Net(nn.Module):
    def __init__(self , print_shape=False):
        self.print_shape = print_shape
        dropoutr = 0.2
        super(Net, self).__init__()
        self.layers1 = nn.ModuleList( [
        nn.Conv2d(9, 64, (3,3)),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)),
        nn.Dropout(dropoutr),
        ResBlock(64,64, (3,3), 1, 1, dropoutr),
        #ResBlock(64,64, (3,3), 1, 1, dropoutr),
        ProjectedResBlock(64,128, (3,3), 1, 1, dropoutr),
        ResBlock(128,128, (3,3), 1, 1, dropoutr),
        ResBlock(128,128, (3,3), 1, 1, dropoutr),
        ProjectedResBlock(128,256, (3,3), 1, 1, dropoutr),
        #(256,256, (3,3), 1, 1, dropoutr),
        ResBlock(256,256, (3,3), 1, 1, dropoutr),
        ProjectedResBlock(256,512, (3,3), 1, 0, dropoutr),
        ResBlock(512,512, (3,3), 1, 1, dropoutr),
        ResBlock(512,512, (3,3), 1, 1, dropoutr),
        nn.Conv2d(512, 512, kernel_size = (2,2), stride = (2,2)),
        nn.BatchNorm2d(512),
        nn.Dropout(dropoutr)])

        self.layers = nn.ModuleList([
        nn.Linear(4096, 2048),
        nn.BatchNorm1d(2048),
        nn.ReLU(),
        nn.Dropout(dropoutr),
        nn.Linear(2048, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Dropout(dropoutr),
        nn.Linear(1024, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Dropout(dropoutr),
        nn.Linear(1024, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(dropoutr),
        nn.Linear(512, 8)])

        for l in self.layers1:
            if isinstance(l, (nn.BatchNorm2d, nn.BatchNorm1d)):
                l.weight.data.fill_(1)
                l.bias.data.zero_()


        for l in self.layers:
            if isinstance(l, (nn.BatchNorm2d, nn.BatchNorm1d)):
                l.weight.data.fill_(1)
                l.bias.data.zero_()
            elif isinstance(l, nn.Conv2d):
                n = l.kernel_size[0] * l.kernel_size[1] * l.out_channels
                l.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(l, nn.Linear):
                nn.init.xavier_normal(l.weight)

    def forward(self, x):
        x = x[:,0,:,:,:]
        for l in self.layers1:
            x = l(x)
            if  (isinstance(l , ProjectedResBlock) or  isinstance(l , ResBlock) or  isinstance(l , nn.Dropout) ) and self.print_shape:
                print(x.shape)
        x = torch.flatten(x, 1)
        for l in self.layers:
            x = l(x)
            if isinstance(l , nn.Dropout) and self.print_shape:
                print(x.shape)
        return x

In [ ]:
net = Net().to(dev)
criterion = nn.CrossEntropyLoss().to(dev)


<ipython-input-12-6582b3845629>:60: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(l.weight)


In [ ]:
a,b = next(iter(trainloader))
a = a.float().to(dev)
r = net.to(dev)
r = net(a).to(dev)
b = b.to(dev)
z = torch.argmax(torch.softmax(r,1), axis=1).to(dev)
print(torch.softmax(r,1))
print(z)
print(b - z)
# print(r.data)
# print(b)


tensor([[0.0907, 0.0687, 0.2542, 0.3140, 0.0717, 0.0349, 0.0279, 0.1379],
        [0.0313, 0.0194, 0.1212, 0.4313, 0.1014, 0.0125, 0.0232, 0.2596],
        [0.0634, 0.0918, 0.0924, 0.2115, 0.1392, 0.0586, 0.0823, 0.2607],
        [0.0623, 0.1045, 0.2170, 0.0388, 0.0245, 0.0483, 0.2035, 0.3011],
        [0.1364, 0.0169, 0.0619, 0.5667, 0.1413, 0.0032, 0.0321, 0.0415],
        [0.1693, 0.0135, 0.1046, 0.4278, 0.0100, 0.0017, 0.0929, 0.1801],
        [0.0770, 0.0774, 0.3188, 0.2883, 0.0221, 0.0382, 0.0139, 0.1644],
        [0.1474, 0.0794, 0.1876, 0.2647, 0.0685, 0.0247, 0.0174, 0.2102],
        [0.0346, 0.1357, 0.1609, 0.2510, 0.0368, 0.0089, 0.1186, 0.2535],
        [0.0457, 0.1057, 0.1506, 0.2021, 0.0531, 0.0874, 0.2193, 0.1361],
        [0.0300, 0.0123, 0.0818, 0.1240, 0.0885, 0.0192, 0.1765, 0.4678],
        [0.0692, 0.0325, 0.0212, 0.5973, 0.0252, 0.0349, 0.0157, 0.2039],
        [0.0078, 0.1015, 0.1745, 0.4299, 0.0651, 0.0250, 0.0649, 0.1314],
        [0.1270, 0.0163, 0.1093, 0.317

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=0.00001)
# exp_lr_scheduler =  torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)


In [ ]:
for epoch in range(50):  # loop over the dataset multiple times
    running_loss = 0.0
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=0)
    for i, data in enumerate(trainloader, 0):
      try:
        inputs, labels = data
        inputs = inputs.float().to(dev)
        labels = labels.to(dev)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
        if i % 10000 == 9999:    # print every 2000 mini-batches
               torch.save(net, "/content/drive/MyDrive/resnet2.pt")
      except Exception as e:
        print(e)



print('Finished Training')

In [ ]:
torch.save(net, "/content/drive/MyDrive/resnet2.pt")

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for i in range(100):
        data = next(iter(trainloader))
        images, labels = data
        images = images.float().to(dev)
        labels = labels.to(dev)
        outputs = net(images)
        predicted = torch.argmax(torch.softmax(outputs.data, 1),axis =1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 73 %
